## General workflow for the analysis of the predictions of the SyntaxSQL
### The spider evaluation scripts only prints out the wrong queries. 
### As the authors have made available the output of the evaluation script, which contains the wrongly predicted queries. 
### IT is possible to find how many/which queries are different from the our gold development set. Which comprises 4 databases.

## Open the dev results
### We changed the path twice to the augmented version of syntax sql and the normal baseline, and reruned the code to find out the % of correct queries.

In [373]:
import pandas as pd 
data = pd.read_csv("/Users/yuriy/Downloads/generated_datasets/generated_data_augment/saved_models/dev_result_eval.txt", sep="\t", header=None)
data


,0
0,"medium pred: select Song_Name,Country,Name from singer order by Age desc"
1,"medium gold: SELECT name , country , age FROM singer ORDER BY age DESC"
2,"medium pred: select Song_Name,Name,Country from singer order by Age desc"
3,"medium gold: SELECT name , country , age FROM singer ORDER BY age DESC"
4,"medium pred: select avg(Age),min(Age),max(Age) from singer where Age = (select max(Age) from singer)"
...,...
1509,group 0.583 0.455 0.707 0.671 0.565
1510,order 0.600 0.462 0.672 0.795 0.643
1511,and/or 0.978 0.924 0.953 0.912 0.940
1512,IUEN 1.000 1.000 0.074 1.000 0.041


## Slice the data to the correct row, uncoment the cell for the augmented version 

In [374]:
filtered_data= data[:1477]# <---- For the augmented version SyntaxSQLAugmented
#filtered_data= data[:1621] # <------- For the normal version of SyntaxSQL

In [375]:
filtered_data

,0
0,"medium pred: select Song_Name,Country,Name from singer order by Age desc"
1,"medium gold: SELECT name , country , age FROM singer ORDER BY age DESC"
2,"medium pred: select Song_Name,Name,Country from singer order by Age desc"
3,"medium gold: SELECT name , country , age FROM singer ORDER BY age DESC"
4,"medium pred: select avg(Age),min(Age),max(Age) from singer where Age = (select max(Age) from singer)"
...,...
1472,"medium gold: SELECT T2.feature_type_name FROM Other_Available_Features AS T1 JOIN Ref_Feature_Types AS T2 ON T1.feature_type_code = T2.feature_type_code WHERE T1.feature_name = ""AirCon"""
1473,medium pred: select property_type_description from Ref_Property_Types
1474,medium gold: SELECT T2.property_type_description FROM Properties AS T1 JOIN Ref_Property_Types AS T2 ON T1.property_type_code = T2.property_type_code GROUP BY T1.property_type_code
1475,hard pred: select property_name from Properties where agreed_selling_price > 'terminal'


## Open the preselected databases of the development set


In [376]:
dev_gold = pd.read_csv("/Users/yuriy/Desktop/Neural-Rule-Based-model-for-Text-to-SQL/data/devPreSelected.csv", index_col=0)
dev_gold

,database,question,sql
0,poker_player,How many poker players are there?,SELECT count(*) FROM poker_player
1,poker_player,Count the number of poker players.,SELECT count(*) FROM poker_player
2,poker_player,List the earnings of poker players in descending order.,SELECT Earnings FROM poker_player ORDER BY Earnings DESC
3,poker_player,"What are the earnings of poker players, ordered descending by value?",SELECT Earnings FROM poker_player ORDER BY Earnings DESC
4,poker_player,List the final tables made and the best finishes of poker players.,"SELECT Final_Table_Made , Best_Finish FROM poker_player"
...,...,...,...
115,singer,What are the names of the singers that have more than one songs?,SELECT T1.Name FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID = T2.Singer_ID GROUP BY T1.Name HAVING COUNT(*) > 1
116,singer,Show the names of singers and the total sales of their songs.,"SELECT T1.Name , sum(T2.Sales) FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID = T2.Singer_ID GROUP BY T1.Name"
117,singer,"For each singer name, what is the total sales for their songs?","SELECT T1.Name , sum(T2.Sales) FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID = T2.Singer_ID GROUP BY T1.Name"
118,singer,List the name of singers that do not have any song.,SELECT Name FROM singer WHERE Singer_ID NOT IN (SELECT Singer_ID FROM song)


## Filter the data from the wrong predicitions so there is only gold values, as we can compare them to the gold values of our development set and then count how many are missing (which would be the correct ones, as the evaluation script of spider only show the wrong queries)

In [377]:
filtered_data = filtered_data[0].str.split(":", n=1, expand=True)
filtered_data.columns = ['Column1', 'Column2']
filtered_data

,Column1,Column2
0,medium pred,"select Song_Name,Country,Name from singer order by Age desc"
1,medium gold,"SELECT name , country , age FROM singer ORDER BY age DESC"
2,medium pred,"select Song_Name,Name,Country from singer order by Age desc"
3,medium gold,"SELECT name , country , age FROM singer ORDER BY age DESC"
4,medium pred,"select avg(Age),min(Age),max(Age) from singer where Age = (select max(Age) from singer)"
...,...,...
1472,medium gold,"SELECT T2.feature_type_name FROM Other_Available_Features AS T1 JOIN Ref_Feature_Types AS T2 ON T1.feature_type_code = T2.feature_type_code WHERE T1.feature_name = ""AirCon"""
1473,medium pred,select property_type_description from Ref_Property_Types
1474,medium gold,SELECT T2.property_type_description FROM Properties AS T1 JOIN Ref_Property_Types AS T2 ON T1.property_type_code = T2.property_type_code GROUP BY T1.property_type_code
1475,hard pred,select property_name from Properties where agreed_selling_price > 'terminal'


## Select only the rows that has gold in the description of the query

In [378]:
gold_values = filtered_data[filtered_data['Column1'].str.contains('gold')]['Column2']
gold_values_df = pd.DataFrame(gold_values, columns=['Column2'])
gold_values_df

,Column2
1,"SELECT name , country , age FROM singer ORDER BY age DESC"
3,"SELECT name , country , age FROM singer ORDER BY age DESC"
5,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'"
7,"SELECT song_name , song_release_year FROM singer ORDER BY age LIMIT 1"
9,SELECT song_name FROM singer WHERE age > (SELECT avg(age) FROM singer)
...,...
1468,SELECT Citizenship FROM singer WHERE Birth_Year < 1945 INTERSECT SELECT Citizenship FROM singer WHERE Birth_Year > 1955
1470,SELECT Citizenship FROM singer WHERE Birth_Year < 1945 INTERSECT SELECT Citizenship FROM singer WHERE Birth_Year > 1955
1472,"SELECT T2.feature_type_name FROM Other_Available_Features AS T1 JOIN Ref_Feature_Types AS T2 ON T1.feature_type_code = T2.feature_type_code WHERE T1.feature_name = ""AirCon"""
1474,SELECT T2.property_type_description FROM Properties AS T1 JOIN Ref_Property_Types AS T2 ON T1.property_type_code = T2.property_type_code GROUP BY T1.property_type_code


## Now the task is to force the comparison between sets of queries, here we lower every string and remove empty spaces

In [379]:
gold_values_df["Column2"] = gold_values_df["Column2"].str.strip().str.lower()
dev_gold["sql"] = dev_gold["sql"].str.strip().str.lower()
gold_values_df

,Column2
1,"select name , country , age from singer order by age desc"
3,"select name , country , age from singer order by age desc"
5,"select avg(age) , min(age) , max(age) from singer where country = 'france'"
7,"select song_name , song_release_year from singer order by age limit 1"
9,select song_name from singer where age > (select avg(age) from singer)
...,...
1468,select citizenship from singer where birth_year < 1945 intersect select citizenship from singer where birth_year > 1955
1470,select citizenship from singer where birth_year < 1945 intersect select citizenship from singer where birth_year > 1955
1472,"select t2.feature_type_name from other_available_features as t1 join ref_feature_types as t2 on t1.feature_type_code = t2.feature_type_code where t1.feature_name = ""aircon"""
1474,select t2.property_type_description from properties as t1 join ref_property_types as t2 on t1.property_type_code = t2.property_type_code group by t1.property_type_code


In [380]:
dev_gold

,database,question,sql
0,poker_player,How many poker players are there?,select count(*) from poker_player
1,poker_player,Count the number of poker players.,select count(*) from poker_player
2,poker_player,List the earnings of poker players in descending order.,select earnings from poker_player order by earnings desc
3,poker_player,"What are the earnings of poker players, ordered descending by value?",select earnings from poker_player order by earnings desc
4,poker_player,List the final tables made and the best finishes of poker players.,"select final_table_made , best_finish from poker_player"
...,...,...,...
115,singer,What are the names of the singers that have more than one songs?,select t1.name from singer as t1 join song as t2 on t1.singer_id = t2.singer_id group by t1.name having count(*) > 1
116,singer,Show the names of singers and the total sales of their songs.,"select t1.name , sum(t2.sales) from singer as t1 join song as t2 on t1.singer_id = t2.singer_id group by t1.name"
117,singer,"For each singer name, what is the total sales for their songs?","select t1.name , sum(t2.sales) from singer as t1 join song as t2 on t1.singer_id = t2.singer_id group by t1.name"
118,singer,List the name of singers that do not have any song.,select name from singer where singer_id not in (select singer_id from song)


In [381]:
gold_values_df["Column2"] = gold_values_df["Column2"].str.replace(' ', '')
dev_gold["sql"] = dev_gold["sql"].str.replace(' ', '')

In [382]:
gold_values_df

,Column2
1,"selectname,country,agefromsingerorderbyagedesc"
3,"selectname,country,agefromsingerorderbyagedesc"
5,"selectavg(age),min(age),max(age)fromsingerwherecountry='france'"
7,"selectsong_name,song_release_yearfromsingerorderbyagelimit1"
9,selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)
...,...
1468,selectcitizenshipfromsingerwherebirth_year<1945intersectselectcitizenshipfromsingerwherebirth_year>1955
1470,selectcitizenshipfromsingerwherebirth_year<1945intersectselectcitizenshipfromsingerwherebirth_year>1955
1472,"selectt2.feature_type_namefromother_available_featuresast1joinref_feature_typesast2ont1.feature_type_code=t2.feature_type_codewheret1.feature_name=""aircon"""
1474,selectt2.property_type_descriptionfrompropertiesast1joinref_property_typesast2ont1.property_type_code=t2.property_type_codegroupbyt1.property_type_code


In [383]:
sql_gold= gold_values_df["Column2"].to_list()
sql_gold

['selectname,country,agefromsingerorderbyagedesc',
 'selectname,country,agefromsingerorderbyagedesc',
 "selectavg(age),min(age),max(age)fromsingerwherecountry='france'",
 'selectsong_name,song_release_yearfromsingerorderbyagelimit1',
 'selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)',
 'selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)',
 'selectlocation,namefromstadiumwherecapacitybetween5000and10000',
 'selectname,capacityfromstadiumorderbyaveragedesclimit1',
 'selectname,capacityfromstadiumorderbyaveragedesclimit1',
 'selectcount(*)fromconcertwhereyear=2014oryear=2015',
 'selectcount(*)fromconcertwhereyear=2014oryear=2015',
 'selectt2.name,count(*)fromconcertast1joinstadiumast2ont1.stadium_id=t2.stadium_idgroupbyt1.stadium_id',
 'selectt2.name,count(*)fromconcertast1joinstadiumast2ont1.stadium_id=t2.stadium_idgroupbyt1.stadium_id',
 'selectt2.name,t2.capacityfromconcertast1joinstadiumast2ont1.stadium_id=t2.stadium_idwheret1.year>=2014groupbyt2.stadium_idorder

## Calculate how many queries can be found from the development evaluation of Syntax Sql in our development set. 
### Reminder we are calculating the number of incorrect queries. Because the evaluation script of spider only shows the wrong queries

In [384]:
index_match = []

i = 0
for value in dev_gold['sql']:
    if value in sql_gold:
        index_match.append(i)
    i+=1

index_match

[2,
 3,
 4,
 5,
 10,
 11,
 12,
 13,
 16,
 17,
 20,
 21,
 22,
 23,
 36,
 37,
 41,
 42,
 44,
 46,
 48,
 49,
 50,
 51,
 52,
 56,
 57,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 82,
 83,
 86,
 87,
 88,
 89,
 90,
 91,
 98,
 99,
 100,
 101,
 102,
 103,
 108,
 109,
 114,
 115,
 116,
 117,
 118,
 119]

## Number of wrong queries using the predicitons of dev_result_eval Augmented

In [385]:
print(len(index_match))
# The number of wrong queries for augmented data is 63
# The number of wrong queries for non-augmented data is 69
print(f"Percentage of correct queries is {(len(dev_gold)-len(index_match))/len(dev_gold)*100}%")

63
Percentage of correct queries is 47.5%
